# Ноутбук по отбору признаков №1

[Пример ноутбука в котором идет речь про отбор признаков](https://github.com/RomanSafronenkov/mlcourse.ai/blob/main/jupyter_russian/topic06_features/topic6_feature_engineering_feature_selection_russian.ipynb)

[Sequential Feature Selection с объяснениями](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.feature_selection/#sequentialfeatureselector)

[О том, почему плох встроенный в RandomForest feature_importance](https://explained.ai/rf-importance/)

In [1]:
from abc import ABC, abstractmethod
import copy

import pandas as pd
import numpy as np

from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.inspection import permutation_importance

from lightgbm import LGBMClassifier

import matplotlib.pyplot as plt
%matplotlib inline

import shap

import warnings
warnings.filterwarnings('ignore')

In [2]:
# создадим искусственный датасет, в котором будут информативные признаки, их дубликаты, их комбинации и шумовые признаки
x, y = make_classification(
    n_samples=10000,
    n_features=100,
    n_informative=15,
    n_redundant=5,
    weights=(0.8, 0.2),
    n_classes=2,
    n_repeated=5,
    n_clusters_per_class=4,
    shift=0.8,
    scale=3.0,
    shuffle=False)

# если не ставить параметр shuffle в True, то сначала будут идти информативные признаки, потом комбинации и потом повторы, после них - мусор
# так удобнее оценить как это все работает

In [3]:
%%time
# замерим метрику на кросс валидации
cross_val_score(LGBMClassifier(verbose=-100), x, y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

CPU times: user 12.4 s, sys: 140 ms, total: 12.5 s
Wall time: 6.33 s


0.9565867283810144

In [4]:
class BaseFeatureSelector(ABC):
    def __init__(self, n_folds=5):
        self.n_folds = n_folds
        self.importances = None

    def fit(self, x, y, **importances_kwargs):
        skf = StratifiedKFold(n_splits=self.n_folds, shuffle=True)  # можно число фолдов и прочие параметры

        self.importances = pd.DataFrame({'feature': np.arange(x.shape[1])})

        for i, (train_index, val_index) in enumerate(skf.split(x, y)):
            x_train, y_train = x[train_index], y[train_index]
            x_val, y_val = x[val_index], y[val_index]

            model = LGBMClassifier(max_depth=5, n_estimators=500, learning_rate=0.05, verbose=-100)
            model.fit(x_train, y_train, eval_set=(x_val, y_val))

            imp = self._get_importances_from_model(model, x_val, y_val, **importances_kwargs)

            self.importances[f'importance_{i}'] = imp

    def get_selected_features(self, threshold):
        assert self.importances is not None, 'Сначала нужно обучить, вызвав метод fit'

        # сделаем отдельно для 0 итерации
        imps = self.importances.loc[:, ['feature', 'importance_0']].sort_values('importance_0', ascending=False)  # выберем важности признаков с 0 итерации
        imps['importance_0'] /= imps['importance_0'].sum()
        imps['cumsum'] = imps['importance_0'].cumsum()  # так как мы их отнормировали, может посчитать кумулятивную сумму
        features = imps.loc[imps['cumsum'] <= threshold, 'feature'].tolist()  # возьмем только те признаки, которые по кумулятивной сумме удовлетворяют
        
        best_features = set(features)  # сделаем множество
        for i in range(1, self.n_folds):
            imps = self.importances.loc[:, ['feature', f'importance_{i}']].sort_values(f'importance_{i}', ascending=False)
            imps[f'importance_{i}'] /= imps[f'importance_{i}'].sum()
            imps['cumsum'] = imps[f'importance_{i}'].cumsum()
            features = imps.loc[imps['cumsum'] <= threshold, 'feature'].tolist()

            best_features &= set(features)  # смотрим на пересечения множеств на разных итерациях кросс-валидации

        return list(best_features)
        
    @abstractmethod
    def _get_importances_from_model(self, model, x, y, **kwargs):
        pass

In [5]:
# отбор признаков с использованием встроеного feature_importance
class LGMFeatureSelection(BaseFeatureSelector):
    def __init__(self, n_folds=5):
        super().__init__(n_folds)

    def _get_importances_from_model(self, model, x, y, importance_type='split'):
        return model.booster_.feature_importance(importance_type=importance_type)

In [6]:
%%time
lgm_fs = LGMFeatureSelection()
lgm_fs.fit(x, y, importance_type='split')

CPU times: user 27.6 s, sys: 203 ms, total: 27.8 s
Wall time: 13.9 s


In [7]:
lgm_fs.importances.head(20)

,feature,importance_0,importance_1,importance_2,importance_3,importance_4
0,0,283,281,310,291,277
1,1,330,298,342,363,305
2,2,382,386,438,431,383
3,3,346,330,345,322,364
4,4,427,403,451,401,397
5,5,388,362,389,384,373
6,6,327,302,340,326,328
7,7,381,373,372,379,419
8,8,410,415,411,377,383
9,9,366,308,362,378,369


In [8]:
best_features_lgm_fs = lgm_fs.get_selected_features(threshold=0.5)
len(best_features_lgm_fs)

14

In [9]:
best_features_lgm_fs  # видно, что идут по порядку

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14]

In [10]:
# замерим метрику на кросс валидации с отобранными признаками
cross_val_score(LGBMClassifier(verbose=-100), x[:, best_features_lgm_fs], y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.9598983935857557

In [11]:
# попробуем тоже самое, но с помощью shap values
class ShapFeatureSelection(BaseFeatureSelector):
    def __init__(self, n_folds=5):
        super().__init__(n_folds)

    def _get_importances_from_model(self, model, x, y, 
                                    is_multiclass=False, feature_perturbation='tree_path_dependent'):
        explainer = shap.TreeExplainer(model, feature_perturbation=feature_perturbation)
        shap_values = explainer.shap_values(x)  # for each class, for each instance

        if is_multiclass:
            if isinstance(shap_values, list):
                # if shap_values in list of n_classes np.arrays of shape [n_samples, n_features]
                importances = []
                for cls_ in shap_values:
                    cls_value = np.abs(cls_).mean(axis=0)
                    importances.append(cls_value.reshape(1, -1))
                    
                importances = np.concatenate(importances).mean(axis=0)
            else:
                # if shap_values in np.array of shape [n_samples, n_features, n_classes]
                importances = np.abs(shap_values).mean(axis=(0, 2))
        else:
            if isinstance(shap_values, list):
                # if shap_values in list of n_classes np.arrays of shape [n_samples, n_features]
                importances = np.abs(shap_values[1]).mean(axis=0)
            else:
                # if shap_values in np.array of shape [n_samples, n_features]
                importances = np.abs(shap_values).mean(axis=0)
        return importances

In [12]:
%%time
shap_fs = ShapFeatureSelection()
shap_fs.fit(x, y, is_multiclass=False, feature_perturbation='tree_path_dependent')

CPU times: user 1min 6s, sys: 220 ms, total: 1min 7s
Wall time: 25.1 s


In [13]:
shap_fs.importances.head(20)

,feature,importance_0,importance_1,importance_2,importance_3,importance_4
0,0,0.120331,0.165546,0.158914,0.119959,0.151279
1,1,0.222161,0.231679,0.242651,0.240228,0.217469
2,2,0.669277,0.695947,0.707646,0.577635,0.693196
3,3,0.296354,0.328124,0.328912,0.337846,0.311341
4,4,0.180501,0.167048,0.191106,0.178394,0.186488
5,5,0.184131,0.160086,0.216943,0.180900,0.195159
6,6,0.387192,0.367869,0.424341,0.386944,0.379377
7,7,0.408509,0.421378,0.426364,0.445890,0.423986
8,8,0.469104,0.487378,0.497010,0.458392,0.467936
9,9,0.290803,0.208366,0.263158,0.205376,0.270807


In [14]:
best_features_shap_fs = shap_fs.get_selected_features(threshold=0.7)
len(best_features_shap_fs)

16

In [15]:
best_features_shap_fs

[1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19]

In [16]:
# замерим метрику на кросс валидации с отобранными признаками
cross_val_score(LGBMClassifier(verbose=-100), x[:, best_features_shap_fs], y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.9573755867341873

In [17]:
# самый интуитивно понятный метод, но самый долгий - permutation importance
class PIFeatureSelection(BaseFeatureSelector):
    def __init__(self, n_folds=5):
        super().__init__(n_folds)

    def _get_importances_from_model(self, model, x, y, **permutation_kwargs):
        importances = np.abs(permutation_importance(model, x, y, **permutation_kwargs)['importances_mean'])
        return importances

In [18]:
%%time
pi_fs = PIFeatureSelection()
pi_fs.fit(x, y, scoring='roc_auc', n_jobs=-1, n_repeats=5)

CPU times: user 1min 31s, sys: 584 ms, total: 1min 32s
Wall time: 2min 29s


In [19]:
pi_fs.importances.head(20)

,feature,importance_0,importance_1,importance_2,importance_3,importance_4
0,0,0.007658,0.009207,0.008020,0.005870,0.007356
1,1,0.013441,0.016086,0.016447,0.016244,0.012049
2,2,0.065422,0.051706,0.058040,0.064037,0.061156
3,3,0.027245,0.024070,0.025101,0.025620,0.024216
4,4,0.015680,0.014505,0.018136,0.016353,0.012671
5,5,0.016307,0.013991,0.009891,0.012967,0.009414
6,6,0.022708,0.023358,0.023931,0.024563,0.021520
7,7,0.030663,0.031906,0.023593,0.037504,0.033559
8,8,0.024956,0.035135,0.030942,0.023377,0.020393
9,9,0.019669,0.015248,0.013680,0.018604,0.014154


In [20]:
best_features_pi_fs = pi_fs.get_selected_features(threshold=0.99)
len(best_features_pi_fs)

21

In [21]:
best_features_pi_fs

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 48]

In [22]:
# замерим метрику на кросс валидации с отобранными признаками
cross_val_score(LGBMClassifier(verbose=-100), x[:, best_features_pi_fs], y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.9647061050989001

# Жадные методы отбора

In [23]:
class SequentialForwardFeatureSelection:
    def __init__(self, model, n_folds, k, smooth=False, stratified=True, scoring='roc_auc'):
        self.model = model
        self.n_folds = n_folds
        self.k = k
        self.stratified = True
        self.smooth = smooth
        self.scoring = scoring

    def _find_feature_to_add(self, x, y, cur_subset, set_to_choose_from):
        best_score = float('-inf')
        best_feature = None

        print('*'*30)
        print('Looking for feature to ADD...')

        for feature in set_to_choose_from:
            cur_subset_copy = cur_subset.copy()
            cur_subset_copy += [feature]

            subset_x = x[:, cur_subset_copy]
            if self.stratified:
                cv = StratifiedKFold(n_splits=self.n_folds, shuffle=True)
            else:
                cv = KFold(n_splits=self.n_folds, shuffle=True)
            score = cross_val_score(estimator=self.model, X=subset_x, y=y, cv=cv, scoring=self.scoring).mean()
            if score > best_score:
                best_feature = feature
                best_score = score
                
        print(f'Best score = {best_score:.4f}')
        return best_feature

    def _find_feature_to_delete(self, x, y, cur_subset):
        if self.stratified:
            cv = StratifiedKFold(n_splits=self.n_folds, shuffle=True)
        else:
            cv = KFold(n_splits=self.n_folds, shuffle=True)
        best_score = cross_val_score(estimator=self.model, X=x[:, cur_subset], y=y, cv=cv, scoring=self.scoring).mean()
        worst_feature = None

        print('='*30)
        print('Looking for feature to DELETE...')
        print(f'Best score = {best_score:.4f}')

        for feature in cur_subset:
            cur_subset_copy = cur_subset.copy()
            del cur_subset_copy[cur_subset_copy.index(feature)]

            subset_x = x[:, cur_subset_copy]
            if self.stratified:
                cv = StratifiedKFold(n_splits=self.n_folds, shuffle=True)
            else:
                cv = KFold(n_splits=self.n_folds, shuffle=True)
            score = cross_val_score(estimator=self.model, X=subset_x, y=y, cv=cv, scoring=self.scoring).mean()
            if score > best_score:
                worst_feature = feature
                best_score = score
        return worst_feature

    def fit(self, x, y):
        best_features = []
        set_to_choose_from = list(range(x.shape[1]))

        best_feature = self._find_feature_to_add(x=x, y=y, cur_subset=best_features, set_to_choose_from=set_to_choose_from)
        best_features += [best_feature]
        print(f'Added feature {best_feature}, current subset = {best_features}')
        del set_to_choose_from[set_to_choose_from.index(best_feature)]

        while len(best_features) < self.k:
            best_feature = self._find_feature_to_add(x=x, y=y, cur_subset=best_features, set_to_choose_from=set_to_choose_from)
            best_features += [best_feature]
            print(f'Added feature {best_feature}, current subset = {best_features}')
            del set_to_choose_from[set_to_choose_from.index(best_feature)]

            if len(best_features) > 2 and self.smooth:
                worst_feature = self._find_feature_to_delete(x=x, y=y, cur_subset=best_features)
                if worst_feature == best_feature:
                    break

                if worst_feature is not None:
                    del best_features[best_features.index(worst_feature)]
                    print(f'Deleted feature {worst_feature}, current subset = {best_features}')
                    set_to_choose_from += [worst_feature]

                    while len(best_features) > 2:
                        worst_feature = self._find_feature_to_delete(x=x, y=y, cur_subset=best_features)
                        if worst_feature is None:
                            break
                        del best_features[best_features.index(worst_feature)]
                        print(f'Deleted feature {worst_feature}, current subset = {best_features}')
                        set_to_choose_from += [worst_feature]
        return best_features

In [24]:
%%time
# самый долгий

seq_fs = SequentialForwardFeatureSelection(
    model=LGBMClassifier(max_depth=5, n_estimators=500, learning_rate=0.05, verbose=-100),
    n_folds=5,
    k=20)
best_features_seq_fs = seq_fs.fit(x, y)
len(best_features_seq_fs)

******************************
Looking for feature to ADD...
Best score = 0.6847
Added feature 17, current subset = [17]
******************************
Looking for feature to ADD...
Best score = 0.7158
Added feature 10, current subset = [17, 10]
******************************
Looking for feature to ADD...
Best score = 0.7603
Added feature 12, current subset = [17, 10, 12]
******************************
Looking for feature to ADD...
Best score = 0.7989
Added feature 4, current subset = [17, 10, 12, 4]
******************************
Looking for feature to ADD...
Best score = 0.8329
Added feature 19, current subset = [17, 10, 12, 4, 19]
******************************
Looking for feature to ADD...
Best score = 0.8607
Added feature 9, current subset = [17, 10, 12, 4, 19, 9]
******************************
Looking for feature to ADD...
Best score = 0.8878
Added feature 7, current subset = [17, 10, 12, 4, 19, 9, 7]
******************************
Looking for feature to ADD...
Best score = 0.908

20

In [25]:
sorted(best_features_seq_fs)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [26]:
# замерим метрику на кросс валидации с отобранными признаками
cross_val_score(LGBMClassifier(verbose=-100), x[:, best_features_seq_fs], y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.967245532858651

In [27]:
set(best_features_lgm_fs) & set(best_features_shap_fs), set(best_features_lgm_fs) & set(best_features_pi_fs), set(best_features_pi_fs) & set(best_features_shap_fs)

({1, 2, 3, 4, 6, 7, 8, 9, 12, 13, 14},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14},
 {1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19})

# Но что, если признаков очень много?

In [28]:
x, y = make_classification(
    n_samples=100000, n_features=1000, n_informative=35, n_redundant=7, n_repeated=8, n_clusters_per_class=4, shift=0.8, scale=3.0, shuffle=False)

In [29]:
%%time
lgm_fs = LGMFeatureSelection()
lgm_fs.fit(x, y, importance_type='split')

CPU times: user 18min 52s, sys: 1.78 s, total: 18min 54s
Wall time: 9min 36s


In [30]:
# замерим метрику на кросс валидации
cross_val_score(LGBMClassifier(verbose=-100), x, y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.9619693773393131

In [31]:
lgm_fs.importances.head(20)

,feature,importance_0,importance_1,importance_2,importance_3,importance_4
0,0,202,208,206,224,208
1,1,487,458,423,432,457
2,2,435,414,444,439,465
3,3,341,344,360,343,310
4,4,281,283,265,298,280
5,5,190,191,199,206,190
6,6,458,469,468,413,453
7,7,244,231,224,208,200
8,8,266,305,286,293,292
9,9,244,231,250,247,243


In [32]:
best_features_lgm_fs = lgm_fs.get_selected_features(threshold=0.85)
len(best_features_lgm_fs)

42

In [33]:
# замерим метрику на кросс валидации с отобранными признаками
cross_val_score(LGBMClassifier(verbose=-100), x[:, best_features_lgm_fs], y, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring='roc_auc').mean()

0.9621921054749389

In [34]:
# видимо имеет смысл осторожнее выбирать отсечку, либо объединять результаты нескольких методов